In [1]:
# Imports
import torch
torch.cuda.empty_cache()

from hr_dv2.utils import *
from hr_dv2 import tr
import numpy as np
from PIL import Image
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from torchvision.io import read_video

from typing import Tuple, List, Any, TypedDict, TypeAlias

ArrList: TypeAlias = List[np.ndarray]

IMG_SIZE = 518
PATCH_SIZE  = 14
PATCH_H, PATCH_W = IMG_SIZE // PATCH_SIZE, IMG_SIZE // PATCH_SIZE
FEAT_DIM = 384

H, W = int(438 * 0.5), int(775 * 0.5)
sub_h: int = H % 14
sub_w: int = W % 14
new_h, new_w = H - sub_h, W - sub_w


frame_transform = tr.transforms.Compose(
    [
    tr.transforms.Resize((H, W)),
    tr.transforms.CenterCrop((new_h, new_w)),
    tr.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)


In [2]:
H, W

(219, 387)

In [3]:
raw_video, _, _ = read_video("data/dog_video_half.mp4")
raw_video = raw_video.to(torch.float16)

/home/ronan/Documents/uni_work/phd/ViT_testing/.venv/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [10]:
transformed_frames = []
for frame in raw_video:
    frame = frame.permute((2, 0, 1))
    cropped = frame_transform(frame)
    transformed_frames.append(cropped)

In [15]:
subsample_idxs = [i * (419 // 4) for i in range(5)]
subsample_idxs

[0, 104, 208, 312, 416]

In [ ]:
from hr_dv2 import HighResDV2

shifts = [i for i in range(1, 3)]
net = HighResDV2("dinov2_vits14", 4)
transforms, inv_transforms = tr.get_shift_transforms(shifts, 'Moore')
net.set_transforms(transforms, inv_transforms)

net.cuda()
net.eval()
pass